In [1]:
from __future__ import print_function

import argparse
import os
import sys
import numpy as np
from textacy.datasets.supreme_court import SupremeCourt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import Dropout
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.python.lib.io import file_io
from time import gmtime, strftime
import pickle

Using TensorFlow backend.


In [5]:
import tensorflow as tf
import keras
# tf.compat.v1.Session()
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess = tf.Session(config=tf.ConfigProto(device_count = {'GPU': 0}, log_device_placement=True)) 
# sess = tf.Session(config=config) 
keras.backend.set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [7]:
model_checkPoint_path = '../ModelCheckpoint'
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 32
MAX_NB_WORDS = 170000
EMBEDDING_DIM = 300

In [4]:
# load pre-trained embedding
print('Indexing word vectors ...')
embeddings_index = {}
f = file_io.FileIO('../data/GoogleNews-vectors-negative300.txt', mode='r')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors ...
Found 3000000 word vectors.


In [5]:
print('Processing text dataset')

sc = SupremeCourt()
print(sc.info)

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
issue_codes.sort()
issue_codes = [str(ic) for ic in issue_codes]

labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))

for record in sc.records():
    if record[1]['issue'] == None: # some cases have None as an issue
        labels.append(labels_index['-1'])
    else:
        labels.append(labels_index[record[1]['issue'][:-4]])
    texts.append(record[0])

print('Found %s texts.' % len(texts))
print('Found %s labels.' % len(labels_index))

Processing text dataset
{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
Found 8419 texts.
Found 15 labels.


In [8]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# with tf.device('/gpu:0'):
data = pad_sequences(sequences)

MAX_SEQUENCE_LENGTH = data.shape[1]

labels = to_categorical(np.asarray(labels))
    
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 173087 unique tokens.
Shape of data tensor: (8419, 90018)
Shape of label tensor: (8419, 15)


In [9]:
# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [12]:
def train_model():

    if not os.path.exists(model_checkPoint_path):
        os.makedirs(model_checkPoint_path)

#     MAX_SEQUENCE_LENGTH = 90018

    # split the data into a training set and a validation set
#     with tf.device('/device:GPU:0'):
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=VALIDATION_SPLIT, random_state=42)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=VALIDATION_SPLIT, random_state=42)
    print(x_train)

    def generator():
        while True:
            indices = list(range(len(x_train)))
            imax = len(indices)//BATCH_SIZE
            for i in range(imax):
                list_IDs_temp = indices[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
                yield x_train[list_IDs_temp], y_train[list_IDs_temp]

    def test_generator():
        while True:
            indices = list(range(len(x_test)))
            imax = len(indices)//BATCH_SIZE
            for i in range(imax):
                list_IDs_temp = indices[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
                yield x_test[list_IDs_temp], y_test[list_IDs_temp]

    def val_generator():
        while True:
            indices = list(range(len(x_val)))
            imax = len(indices)//BATCH_SIZE
            for i in range(imax):
                list_IDs_temp = indices[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
                yield x_val[list_IDs_temp], y_val[list_IDs_temp]


    print('Training model.')

    model = Sequential()
    model.add(
      Embedding(num_words,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                input_length=MAX_SEQUENCE_LENGTH,
                trainable=False)
    )
    model.add(Dropout(0.25))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(5))
    model.add(Dropout(0.25))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(5))
    model.add(Dropout(0.25))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(labels_index), activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    
    # os.path.basename(__file__)[:-3]
    checkpointer = ModelCheckpoint(filepath = model_checkPoint_path + 'cnn15' +
        "-{epoch:02d}-{val_acc:.2f}.hdf5",
                                   monitor='val_acc',
                                   verbose=2,
                                   save_best_only=True,
                                   mode='max')

    earlystopper = EarlyStopping(monitor='val_loss',
                             min_delta=0,
                             patience=0,
                             verbose=2,
                             mode='auto')

    model.summary()

    model.fit_generator(generator=generator(),
                        steps_per_epoch = len(x_train)//BATCH_SIZE,
                        epochs=50,
                        verbose=1,
                        validation_data=test_generator(),
                        validation_steps=len(x_test)//BATCH_SIZE,
                        callbacks=[checkpointer, earlystopper],
                        shuffle=True)
    
#     model.fit(x_train, y_train,
#                 steps_per_epoch = len(x_train)//BATCH_SIZE,
#                 epochs=50,
#                 verbose=1,
#                 validation_data=(x_val, y_val),
#                 validation_steps=len(x_test)//BATCH_SIZE,
#                 callbacks=[checkpointer, earlystopper],
#                 shuffle=True)
    
    score = model.evaluate_generator(val_generator(),
                                     steps=len(x_val)//BATCH_SIZE)

    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Save Keras ModelCheckpoints locally
    model.save('model.hdf5')


In [ ]:
# if __name__ == '__main__':
train_model()

[[    0     0     0 ...  1364     3  2830]
 [    0     0     0 ...  1362  2324  1214]
 [    0     0     0 ...   374     7  2969]
 ...
 [    0     0     0 ...    14 13615   470]
 [    0     0     0 ...    63  2435     4]
 [    0     0     0 ...     8   297   186]]
Training model.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 90018, 300)        51000000  
_________________________________________________________________
dropout_6 (Dropout)          (None, 90018, 300)        0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 90014, 128)        192128    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 18002, 128)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 18